In [19]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [20]:
model = tf.keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False)

In [25]:
ds, info = tfds.load("mnist", with_info=True)
dataset_size = info.splits["train"].num_examples # 3670 
class_names = info.features["label"].names # ["dandelion", "daisy", ...] 
n_classes = info.features["label"].num_classes # 5

test_set, valid_set, train_set = tfds.load("mnist", split=["train[:10%]", "train[10%:25%]", "train[25%:]"])
 
avg = tf.keras.layers.GlobalAveragePooling2D()(model.output) 
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg) 
model = tf.keras.Model(inputs=model.input, outputs=output)

optimizer = tf.keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01) 
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]) 
history = model.fit(train_set, epochs=5, validation_data=valid_set)

KeyError: "Invalid split train[:10%]. Available splits are: ['test', 'train']"

In [15]:
info.splits

{'test': <tfds.core.SplitInfo num_examples=10000>,
 'train': <tfds.core.SplitInfo num_examples=60000>}